## Homework: Introduction
In this homework, we'll learn more about search and use Elastic Search for practice.

### Q1. Running Elastic
Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [5]:
!curl -X GET http://localhost:9200/

{
  "name" : "8afb23dd812f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "-hCY_hiXTCK1xA2_ygov3g",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### Getting the data
Now let's get the FAQ data. You can run this snippet:

In [3]:
!pip install requests --quiet

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the `course` field a keyword; the rest should be text.

Don't forget to install the ElasticSearch client for Python: 
```python
pip install elasticsearch
```

Which function do you use to add your data to `Elastic`?
- insert
- index
- put
- add

In [6]:
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

index_name = "course-questions"
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1, 
        "number_of_replicas": 0 
    },
    "mappings": { 
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
## Printing one document
print(documents[0])

## Indexing the documents in the ElasticSearch
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


  0%|          | 0/948 [00:00<?, ?it/s]

### Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?
- 84.50
- 64.50
- 44.50
- 24.50

Look at the `_score` field.

In [21]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
        "size": 5,  # Limit the search to return only the top 5 results
        "query": {
            "bool": {
                # 'must' clause: defines conditions that must match
                "must": {
                    "multi_match": {
                        "query": query,  # The user's input query string

                        # Search across multiple fields in the index
                        "fields": [
                            "question^4",  # Boost the 'question' field by 3x to increase its weight in scoring
                            "text"
                        ],
                        "type": "best_fields"  # Use the best matching field to score each document
                    }
                },
                # 'filter' clause: restrict results without affecting score
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"  # Only include results from this specific course
                    }
                }
            }
        }
    }
# Send the query to Elasticsearch and store the response
response = es_client.search(index=index_name, body=search_query)
response["hits"]['hits'][0]['_score']

31.973522

### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?
- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?

In [28]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
        "size": 3,  # Limit the search to return only the top 5 results
        "query": {
            "bool": {
                # 'must' clause: defines conditions that must match
                "must": {
                    "multi_match": {
                        "query": query,  # The user's input query string

                        # Search across multiple fields in the index
                        "fields": [
                            "question^4",  # Boost the 'question' field by 3x to increase its weight in scoring
                            "text"
                        ],
                        "type": "best_fields"  # Use the best matching field to score each document
                    }
                },
                # 'filter' clause: restrict results without affecting score
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"  # Only include results from this specific course
                    }
                }
            }
        }
    }
# Send the query to Elasticsearch and store the response
response = es_client.search(index=index_name, body=search_query)
response["hits"]['hits'][-1]["_source"]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks `(\n\n)`

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the len function)
- 946
- 1446
- 1946
- 2446

In [69]:
def elastic_search(query):
    # Construct the Elasticsearch query object
    search_query = {
        "size": 3,  # Limit the search to return only the top 5 results
        "query": {
            "bool": {
                # 'must' clause: defines conditions that must match
                "must": {
                    "multi_match": {
                        "query": query,  # The user's input query string

                        # Search across multiple fields in the index
                        "fields": [
                            "question^4",  # Boost the 'question' field by 3x to increase its weight in scoring
                            "text"
                        ],
                        "type": "best_fields"  # Use the best matching field to score each document
                    }
                },
                # 'filter' clause: restrict results without affecting score
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"  # Only include results from this specific course
                    }
                }
            }
        }
    }
    result_docs = []  # Prepare a list to collect the final matched documents
    # Iterate over each hit in the search result
    for hit in response['hits']['hits']:
        # '_source' contains the actual content of the document
        result_docs.append(hit['_source'])
    # Return the list of matched documents (each is a dict with question, text, section, and course)
    return result_docs

In [70]:
def build_prompt(query, search_results):
    """
    Builds a prompt string for the LLM using retrieved FAQ entries as context.

    Args:
        query (str): The user's natural language question.
        search_results (list): A list of relevant documents returned by `search()`.

    Returns:
        str: A formatted prompt to send to the LLM.
    """

    # Define the context template (Q/A format)
    context_template = "Q: {question}\nA: {text}"

    # Format each search result using the context template
    context_entries = [
        context_template.format(question=doc['question'], text=doc['text'])
        for doc in search_results
    ]

    # Join entries with two line breaks
    context = "\n\n".join(context_entries)

    # Define the overall prompt template
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    # Fill in the final prompt
    prompt = prompt_template.format(question=query, context=context)

    return prompt

In [71]:
query = "How do I execute a command in a running docker container?"
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
print(f"Length of the Prompt is: {len(prompt)}")

Length of the Prompt is: 1101


### Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:
```python
pip install tiktoken
```

Let's calculate the number of tokens in our query:
```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the encode function. How many tokens does our prompt have?
- 120
- 220
- 320
- 420

Note: to decode back a token into a word, you can use the decode_single_token_bytes function:
```python
encoding.decode_single_token_bytes(63842)
```

In [72]:
!pip install tiktoken --quiet

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

242

### Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

### Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:
- Input: $0.005 / 1K tokens
- Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.